# Agrupación y Segmentación de Vecindarios en Toronto
## Parte 1

1. Empiece por crear un nuevo Notebook para esta tarea.
2. Use el Cuaderno para crear el código para rastrear la siguiente página de Wikipedia, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, para obtener los datos que están en la tabla de códigos postales y transformar los datos en un marco de datos de pandas como el que se muestra a continuación:

Importamos las librerías que vamos a utilizar.
Después usamos "wget" para descargarnos la página de wikipedia de Vecindarios de Toronto.

In [1]:
#Importamos la librería "NumPy" como 'np'
#Importamos la librería "Pandas" como 'pd'
#Importamos la librería "Codecs"
#Importamos la librería "BeautifulSoup" desde "bs4"
import numpy as np 
import pandas as pd
import codecs
from bs4 import BeautifulSoup
#Usamos wget para descargar la información de la web
!wget https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print("✅  Lista de códigos postales descargada ✅")


--2022-12-28 12:38:13--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolviendo en.wikipedia.org (en.wikipedia.org)... 185.15.58.224
Conectando con en.wikipedia.org (en.wikipedia.org)[185.15.58.224]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: no especificado [text/html]
Grabando a: «List_of_postal_codes_of_Canada:_M.1»

List_of_postal_code     [  <=>               ]  89,40K   372KB/s    en 0,2s    

2022-12-28 12:38:13 (372 KB/s) - «List_of_postal_codes_of_Canada:_M.1» guardado [91550]

✅  Lista de códigos postales descargada ✅


In [2]:
# Decodificamos el archivo y asignamos el resultado a "archivo"
archivo = codecs.open('List_of_postal_codes_of_Canada:_M', encoding='utf-8')
# Leemos el archivo y lo asignamos a la variable "html"
html = archivo.read()
# Creamos un objeto "bsoup" y parseamos el archivo leído previamente
bsoup = BeautifulSoup(html, 'html.parser')
# Asignamos la tabla a un "DataFrame"
df = (pd.read_html(str(bsoup.table))[0])

3. Para crear el marco de datos anterior:

●      El marco de datos constará de tres columnas: CódigoPostal, Municipio y Vecindario

    ● Leemos el archivo y lo asignamos a un "string".

    ● Utilizaremos la librería "Beautiful Soup" para parsear la tabla de vecindarios a un objeto.

    ● Cogemos sólo la sección de la tabla y la asignamos como DataFrame.

    ● Renombramos las columnas.



In [3]:
# Asignamos los nombres de las columnas requeridos
column_names = ['CódigoPostal', 'Municipio', 'Vecindario']
# Creamos el DataFrame Final
dfFinal = pd.DataFrame(columns = column_names)

postalCode = []
borough = []
neighborhood = []
# Recorremos el dataframe celda por celda y asignamos en 'dfFinal' los datos ordenados
for index, row in df.iterrows():
    for cell in row:
        postalCode.append(cell[:3])
        borough.append(cell[3:cell.find('(')])
        neighborhood.append(cell[cell.find('(') + 1:cell.find(')')])

dfFinal['CódigoPostal'] = postalCode
dfFinal['Municipio'] = borough
dfFinal['Vecindario'] = neighborhood

dfFinal

,CódigoPostal,Municipio,Vecindario
0,M1A,Not assigne,M1ANot assigne
1,M2A,Not assigne,M2ANot assigne
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigne,M5ZNot assigne
176,M6Z,Not assigne,M6ZNot assigne
177,M7Z,Not assigne,M7ZNot assigne
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...



●      Procese únicamente las celdas que tengan un municipio asignado. Ignore las celdas con un municipio que esté No asignado.

    ● Eliminamos las celdas no asignadas para limpiar los datos.

In [4]:
# Nos deshacemos de los Municipios no asignados
dfFinal = dfFinal[dfFinal['Municipio'] != "Not assigne"].reset_index(drop=True)

●      Puede existir más de un vecindario en un área de código postal. Por ejemplo, en la tabla de la página de Wikipedia, notará que M5A aparece dos veces y tiene dos vecindarios: Harbourfront y Regent Park. Estas dos filas se combinarán en una fila con los vecindarios separados con una coma como se muestra en la fila 11 en la tabla anterior.

    No hay vecindarios no asignados, por lo tanto no es necesario añadir código para resolverlo en el momento de hacer este notebook.
    No obstante, aquí tenemos cómo se ejecutaría.

In [5]:
na = dfFinal[dfFinal['Vecindario']=="Not assigned"]
for index in range(len(na)):
    r = na.iloc[index]
    r['Vecindario']=r['Municipio']
    dfFinal.iloc[na.index[0]]['Vecindario'] = r['Municipio']

dfFinal.head(10)   

,CódigoPostal,Municipio,Vecindario
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


●      Si una celda tiene un municipio, pero un vecindario No asignado, entonces el vecindario será el mismo que el municipio.

In [6]:
aggregations = {
    'Vecindario':lambda x:'/ '.join(x)}
gb = dfFinal.groupby('CódigoPostal').agg(aggregations).reset_index()
x = dfFinal.drop('Vecindario',axis=1)
m = pd.merge(x, gb, on='CódigoPostal', how='left')
noDuplicated = m.drop_duplicates()
noDuplicated.to_csv("Toronto_Q1.csv", index=False)
noDuplicated

,CódigoPostal,Municipio,Vecindario
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...



●      Limpie su Notebook y agregue celdas de Anotación para explicar su trabajo y cualquier suposición que esté haciendo.

●      En la última celda de su notebook, use el método .shape para imprimir el número de filas de su marco de datos.

In [7]:
noDuplicated.shape

(103, 3)

4. Envíe un enlace a su notebook en su repositorio de Github. (10 puntos)

Nota: Existen diferentes bibliotecas y paquetes de raspado de sitios web en Python. Para indagar minuciosamente la tabla anterior, simplemente puede usar pandas para leer la tabla en un marco de datos de pandas.

Otra forma, que ayudaría a aprender para casos más complicados de rastreo web es usando el paquete BeautifulSoup. Aquí está la página de documentación principal del paquete: http://beautiful-soup-4.readthedocs.io/en/latest/

Use pandas, o el paquete BeautifulSoup, o cualquier otra forma con la que se sienta cómodo para transformar los datos de la tabla en la página de Wikipedia en el marco de datos de pandas anterior.